In [1]:
import sys
if 'analysis' not in sys.modules:
    sys.path.append('../') 

In [2]:
from analysis import DumpAnalyzer
from os import path, walk, mkdir
from math import exp
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp
import lzma
import tqdm

In [3]:
# Settings

# Directory where the data is stored
# A file example would be ../analysis_many_dims/run_N/TEMP_XD_Y_Sigma.txt
# The file name has to be in the format TEMP_XD_Y_Sigma.txt (.xz is also supported)
# Where TEMP is the temperature, X is the number of dimensions, Y is (counterintuitively) the x value where it is going to be plotted
# This is because right now, Y is the saddle sigma value
files_dir = path.join('..', 'analysis_many_dims')

# r value threshold for rejecting data
r_threshold = 0.98

# Number of cores to use
# Set to None to use all cores, if the value is greater than the number of cores available, it will use all cores
cores = 10

# Directory where the plots will be saved
image_dir = path.join('..', 'plots')

# Directory in which images of discarded data will be saved, set to None to disable
# discarded_dir = path.join('..', 'plots','discarded')
discarded_dir = None 

# Set to true if you don't want warnings
silent = True

# If you want to use specific dimensions set this to an iterable of dimensions otherwise set to None
use_dims = None

# Rate prefactor used in the KMC (Will be used to make the data be in the same units as our reference (1e-13))
# Set to None to avoid conversion
rate_prefactor = 1e-14

# Error bar function
# possible values are 'std' for standard deviation, 'sem' for standard error of the mean
# or a function that takes a list of values and returns the error bar
err_bar = 'sem'

# Units for the diffusivity
# Set to 'cm' for cm^2/s or 'angstrom' for angstrom^2/s
# It will be used to convert the diffusivity to the correct units
# It does assume that the analysis was done in angstrom and seconds
distance_units='cm'

In [4]:
REFERENCE_PREFACTOR = 1e-13 # Reference prefactor used in the KMC
CM_2_IN_ANGSTROM_2 = 1e-16 # 1 cm^2 = 1e-16 angstrom^2
BOLTZMANN_CONSTANT = 8.6173303e-5 # eV/K

if err_bar == 'std':
    err_bar = np.std
elif err_bar == 'sem':
    err_bar = lambda lis: np.std(lis, ddof=1)/np.sqrt(len(lis))

if not callable(err_bar):
    raise ValueError('err_bar must be a function or one of the strings "std" or "sem"')

if cores > mp.cpu_count() or cores is None:
    cores = mp.cpu_count()
if cores < 1:
    cores = 1

# Creating the directories if they don't exist
if not path.exists(image_dir):
    print(f'Creating directory {image_dir}')
    mkdir(image_dir)
if discarded_dir is not None and not path.exists(discarded_dir):
    print(f'Creating directory {discarded_dir}')
    mkdir(discarded_dir)

# Making use_dims an iterable if it is not already
if use_dims is not None and not hasattr(use_dims, '__iter__'):
    use_dims = [use_dims]

In [13]:
def process_file(fp):
    # file_path example would be ../analysis_many_dims/run_N/TEMP_XD_Y_Sigma.txt
    file_name = path.basename(fp)
    # file_name example would be TEMP_XD_Y_Sigma.txt
    # split the file name into the parts we need
    parts = file_name.split('_')
    # parts example would be ['TEMP', 'XD', 'Y', 'Sigma.txt']
    # get the dimensions and sigma from the parts
    d = int(parts[1][:-1])
    s = float(parts[2])
    if file_name.endswith('.xz'):
        with lzma.open(fp, 'rt') as f:
            lyz = DumpAnalyzer(import_data_file=f)
    else:
        with open(fp, 'r') as f:
            lyz = DumpAnalyzer(import_data_file=f)
    r = lyz.fit_line(step_size=100)
    dif = lyz.line_of_best_fit[0] / (2 * d)
    if r < r_threshold:
        if not silent:
            print(f'Warning: R value for {fp} is {r}, rejecting data.')
        if discarded_dir is None:
            return None
        std = lyz.standard_deviation
        lyz.standard_deviation = None
        run_N = path.basename(path.dirname(fp))
        lyz.generate_plot(output_filename=path.join(discarded_dir, f'{run_N}_{file_name}.png'))
        lyz = None
    del lyz # Delete the lyz object to free up memory
    return d, s, dif

diffusivity = {} # diffusivity[dims][sigma] = [] of diffusivity
means = {} # means[dims][sigma] = mean of diffusivity
error_bars = {} # error_bars[dims][sigma] = error_bars of the diffusivity

paths = []
for root, _, files in walk(files_dir):
    if len(files) == 0:
        continue
    for file in files:
        parts = file.split('_')
        if use_dims is not None and int(parts[1][:-1]) not in use_dims:
            continue
        if file.endswith('.csv') or file.endswith('.txt') or file.endswith('.xz'):
            file_path = path.join(root, file)
            paths.append(file_path)

del files, file, parts, file_path, root

# Multiprocessing to make the analyzers
print(f'Using {cores} cores')
with mp.Pool(cores) as pool, tqdm.tqdm(total=len(paths)) as pbar:
    def update(*a):
        pbar.update()
    pbar.set_description('Processing files')
    pbar.unit = 'files'
    runs_list = [pool.apply_async(process_file, args=(fp,), callback=update) for fp in paths]
    runs_list = [r.get() for r in runs_list]
    for i in range(len(runs_list)):
        retval = runs_list[i]
        if retval is None:
            continue
        dim, sig, diff = retval
        if dim not in diffusivity:
            diffusivity[dim] = {}
            means[dim] = {}
            error_bars[dim] = {}
        if sig not in diffusivity[dim]:
            diffusivity[dim][sig] = []
        diffusivity[dim][sig].append(diff)


In [6]:
for dims in diffusivity:
    for sigma in diffusivity[dims]:
        conversion_factor = (REFERENCE_PREFACTOR / rate_prefactor) if rate_prefactor is not None else 1
        conversion_factor *= CM_2_IN_ANGSTROM_2 if distance_units == 'cm' else 1
        diffusivity[dims][sigma] = np.array(diffusivity[dims][sigma]) * conversion_factor
        means[dims][sigma] = np.mean(diffusivity[dims][sigma])
        error_bars[dims][sigma] = err_bar(diffusivity[dims][sigma])

In [7]:
# Checking that there are at least 10 data points for each sigma
for dims in diffusivity:
    for sigma in diffusivity[dims]:
        if len(diffusivity[dims][sigma]) < 10:
            print(f'Warning: Not enough data points for {dims}D and sigma {sigma}, only {len(diffusivity[dims][sigma])} data points.')

In [18]:

# graph dims number of lines with x being sigma and y being the diffusivity, with error bars
# sort the means and error_bars based on dims
dims_sorted = sorted(means.keys())

def one_d_analytical(s):
    barrier_mu = (-8494.804656093489) - (-8495.936351)
    temp = 1000
    return exp(-barrier_mu/(BOLTZMANN_CONSTANT*temp)- s**2/((BOLTZMANN_CONSTANT*temp)**2)) /8

for dims in dims_sorted:
    # Analytical solutions is known for 1D, so we will plot the known solution as well
    plt.yscale('linear')
    x = list(means[dims].keys())
    y = list(means[dims].values())
    yerr = list(error_bars[dims].values())
    # Sort x,y,yerr based on x
    # Use numpy to sort the x values and then use the same indices to sort the y and yerr values
    x = np.array(x)
    y = np.array(y)
    yerr = np.array(yerr)
    sort_indices = np.argsort(x)
    x = x[sort_indices]
    y = y[sort_indices]
    yerr = yerr[sort_indices]
    plt.errorbar(x, y, yerr=yerr, label=f'{dims}D')
    if dims == 1:
        y_analytical = [one_d_analytical(s) for s in x]
        plt.plot(x, y_analytical, label='Analytical')
    plt.xlabel('Sigma')
    plt.ylabel('Diffusivity')
    plt.legend()
    plt.savefig(path.join(image_dir, f'diffusivity_{dims}D.png'))
    # Clear the plot for the next set of data
    plt.clf()


In [9]:
# Do the same as last except all in one graph
# graph dims number of lines with x being sigma and y being the diffusivity, with error bars
# sort the means and error_bars based on dims
dims_sorted = sorted(means.keys())

for dims in dims_sorted:
    x = list(means[dims].keys())
    y = list(means[dims].values())
    yerr = list(error_bars[dims].values())
    # Sort x,y,yerr based on x
    # Use numpy to sort the x values and then use the same indices to sort the y and yerr values
    x = np.array(x)
    y = np.array(y)
    yerr = np.array(yerr)
    sort_indices = np.argsort(x)
    x = x[sort_indices]
    y = y[sort_indices]
    yerr = yerr[sort_indices]
    plt.errorbar(x, y, yerr=yerr, label=f'{dims}D')

plt.xlabel('Sigma')
plt.ylabel('Diffusivity')
plt.legend()
plt.savefig(path.join(image_dir, 'diffusivity_all_dims.png'))
# Clear the plot for the next set of data
plt.clf()

In [12]:
# Graph 5d and compare with 5d stored in ../temp/5d.txt
# ../temp/5d.txt is a file with format "SIGMA 0.0 DIFFUSIVITY ERR"

# Read the file
with open(path.join('..', 'temp', '5d.txt'), 'r') as f:
    conversion_factor = CM_2_IN_ANGSTROM_2 if distance_units == 'cm' else 1
    lines = f.readlines()
    x = []
    y = []
    yerr = []
    for line in lines:
        parts = line.split()
        x.append(float(parts[0]))
        # parts[1] is just a column of 0.0
        y.append(float(parts[2]))
        yerr.append(float(parts[3]))
    x = np.array(x) 
    y = np.array(y) * conversion_factor
    yerr = np.array(yerr)*conversion_factor 
    
    plt.errorbar(x, y, yerr=yerr, label='5D (Stored)')
    # If we have 5D data, plot it
    if 5 in dims_sorted:
        x = list(means[5].keys())
        y = list(means[5].values())
        yerr = list(error_bars[5].values())
        # Sort x,y,yerr based on x
        # Use numpy to sort the x values and then use the same indices to sort the y and yerr values
        x = np.array(x)
        y = np.array(y)
        yerr = np.array(yerr)
        sort_indices = np.argsort(x)
        x = x[sort_indices]
        y = y[sort_indices]
        yerr = yerr[sort_indices]
        plt.errorbar(x, y, yerr=yerr, label='5D (Computed)')
    plt.xlabel('Sigma')
    plt.ylabel('Diffusivity')
    plt.legend()
    plt.show()
    plt.clf()